In [2]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
# constrcute the river network topology
folder_to_save = './shp_output/'
extension = ''


In [36]:
if platform.system().lower() == "darwin": # on personal computer, shold be changed accodingly
    shp_riv = gpd.read_file('/Volumes/F:/hydrography/HDMA/processed/hdma_global_stream.shp')
    
elif platform.system().lower() == "linux": # on remote computer, should be changed accordigly
    shp_riv = gpd.read_file('/home/shg096/data/HDMA/processed/hdma_global_stream.shp')
    
ll
shp_cat = pd.DataFrame()
    
for i in np.array([1000000,2000000,3000000,4000000,5000000,6000000]):
    
    # 
    temp = gpd.read_file(folder_to_save+'hdma_global_catch_v2_'+str(i)+'.shp')
    
    shp_cat = pd.concat([shp_cat, temp], axis=0)
    
    
    


In [43]:
295335-278758

16577

In [37]:
print(len(shp_cat))
print(len(shp_riv))

cat_included = shp_cat[shp_cat['hruid'].isin(shp_riv['seg_id'])]
cat_included = cat_included.sort_values(by='hruid')
cat_included = cat_included.reset_index()
cat_included


295335
278758


,index,hruid,PFAF,area_org,geometry
0,54240,1000000,17974322,2.482575e+08,"POLYGON ((145.55819 60.31653, 145.56236 60.316..."
1,20778,1000001,11526422,2.492364e+08,"POLYGON ((75.79417 15.78083, 75.79417 15.77833..."
2,69318,1000002,1953282,2.492635e+08,"POLYGON ((120.84083 29.39333, 120.84333 29.393..."
3,64959,1000003,19421241,2.478547e+08,"MULTIPOLYGON (((118.91583 32.76000, 118.91583 ..."
4,55579,1000004,1826521,2.466977e+08,"POLYGON ((135.25917 45.84083, 135.25917 45.839..."
...,...,...,...,...,...
278753,10378,6021861,6213,7.840996e+07,"POLYGON ((139.77417 -34.03333, 139.77417 -34.0..."
278754,10368,6021862,62117,1.337588e+09,"POLYGON ((139.68833 -34.01750, 139.69667 -34.0..."
278755,10366,6021863,62115,6.422436e+08,"POLYGON ((139.57500 -34.70667, 139.57583 -34.7..."
278756,10364,6021864,62113,3.290790e+08,"POLYGON ((139.39250 -34.86167, 139.39333 -34.8..."


In [ ]:
#shp_riv = shp_riv.sort_values(by='seg_id')
#shp_riv = shp_riv.reset_index(drop=True)
#shp_riv ['area'] = cat_included ['area_org']
shp_riv_temp  = shp_riv[['seg_id','Tosegment','Length','area','Slope']]

shp_riv_temp

In [42]:
# excluded catchements
cat_excluded = shp_cat[~shp_cat['hruid'].isin(shp_riv['seg_id'])]
cat_excluded['Tosegment'] = -9999
cat_excluded['Length'] = 1
cat_excluded['area'] = 1
cat_excluded['Slope'] = 0.0001
cat_excluded['seg_id'] = cat_excluded['hruid']
cat_excluded_temp  = cat_excluded[['seg_id','Tosegment','Length','area','Slope']]
cat_excluded_temp


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,seg_id,Tosegment,Length,area,Slope
0,1073085,-9999,1,1,0.0001
12,1073086,-9999,1,1,0.0001
23,1073087,-9999,1,1,0.0001
27,1073088,-9999,1,1,0.0001
29,1073089,-9999,1,1,0.0001
...,...,...,...,...,...
24633,6024637,-9999,1,1,0.0001
24635,6024638,-9999,1,1,0.0001
24637,6024639,-9999,1,1,0.0001
24639,6024640,-9999,1,1,0.0001


In [49]:
riv_network = pd.concat([shp_riv_temp, cat_excluded_temp], axis=0)
riv_network

riv_network = riv_network.rename(columns={'seg_id': 'ID',
                                          'Tosegment': 'ID_next',
                                          'Length':'length',
                                          'area':'area',
                                          'slope':'slope'})

riv_network

df = riv_network


# check if everything is ok with the network topology, no circular configuration

In [50]:
def get_all_downstream (seg_IDs,
                        down_IDs):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/EASYMORE
    @ author's email id:       sh.gharari@gmail.com
    @ license:                 GNU-GPLv3
    This function get a 1-D array of numpy arrays of river reach ID and a similar 1-D array
    of downstream river reach ID
    Arguments
    ---------
    seg_IDs: the 1D array of seg id [n,]
    down_IDs: the 1D array of downstream seg id [n,]; if no down_IDs for a given segment should be negative
    Returns
    -------
    NTOPO: the 2D array of downsream seg id s [n,10000]; 10000 is maximume number of downstream
    """
    import pandas as pd
    import numpy as np
    #
    seg_IDs = np.array(seg_IDs)
    down_IDs = np.array(down_IDs)
    NTOPO = np.empty([len(seg_IDs),10000]) # create the empty array with length of seg_IDs and 10000
    NTOPO [:] = np.nan # populate with nan
    NTOPO [:,0] = seg_IDs # assign the first colomn as seg id
    # loop over the seg_IDs
    for i in np.arange(len(seg_IDs)):
        ID = seg_IDs [i] # get the seg ID
        # print(ID)
        down_ID = down_IDs [i] # get the seg downstream ID
        if down_ID in seg_IDs: # check if the downstream seg is part of river network
            down_stream_exists = True
        else:
            down_stream_exists = False
        m = 1 # initialize m
        while down_ID > 0 and down_stream_exists: # while not the last segment
            # update the ID and ID down
            idx = np.where (seg_IDs == down_ID) # get the index of the segment that is downstream
            ID = seg_IDs [idx] # update the ID
            down_ID = down_IDs [idx] # update the downstream
            if down_ID in seg_IDs:
                down_stream_exists = True
            else:
                down_stream_exists = False
            NTOPO[i,m] = ID
            m += 1
            if m > 9000:
                print(ID)
                sys.exit('fix the ID it seems that the following ID is in a circular network')
    return NTOPO

seg_IDs        = np.array(df.ID)
down_IDs       = np.array(df.ID_next)

NTOPO          = get_all_downstream (seg_IDs,down_IDs)


In [8]:
# check the number that is under the lake (both river and also the subbasin)
df = pd.read_csv('../data/Network_topology_no_lake.csv')



print(df)


df.to_csv('../data/Network_topology_Doll_param.csv')

df = df.set_index('ID')

ds = df.to_xarray()

ds.attrs['Conventions'] = 'CF-1.6'
ds.attrs['License']     = 'The data were written by Shervan Gharari. They are under GPL.'
ds.attrs['history']     = 'Created ' + time.ctime(time.time())
ds.attrs['source']      = 'Written by test script of utilities (https://github.com/ShervanGharari/utility-codes'

var_info = {'ID':        {'long_name': 'ID',        'unit': '-'},
            'ID_next':   {'long_name': 'ID next',   'unit': '-'},
            'length':    {'long_name': 'length',    'unit': 'm'},
            'area':      {'long_name': 'area',      'unit': 'm**2'},
            'slope':     {'long_name': 'slope',     'unit': 'm m**-1'}}

var_encoding = {'ID':        {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                'ID_next':   {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                'length':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                'area':      {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                'slope':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9}}

for key in var_info.keys():
    for key1 in var_info[key].keys():
        print(var_info[key][key1])
        ds[key].attrs[key1] = var_info[key][key1]
        print(ds[key].attrs[key1])


if os.path.isfile('../data/Network_topology_HDMA_HydroLake_v1.nc'):
    os.remove('../data/Network_topology_HDMA_HydroLake_v1.nc')

ds = ds.rename_dims({'ID': 'n'})
# ds.to_netcdf('../data/Network_topology_HDMA_HydroLake_v1.nc',\
#              encoding = {var: {'_FillValue': -9999, 'zlib': True, 'complevel': 9} for var in ds.data_vars})
ds.to_netcdf('../data/Network_topology_HDMA_HydroLake_v1.nc',\
             encoding = var_encoding)

ds = xr.open_dataset('../data/Network_topology_HDMA_HydroLake_v1.nc')

ds


number of river segments and cat with length and area of zero:  1186
number of river segments with length of zero:  25155
number of cat with area of zero:  0


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

             ID  ID_next      length          area  lake  endorheic  slope  \
0       1000000  1035282  616.342661  2.482575e+08     0          0      0   
1       1000001  1028025  192.229875  2.492364e+08     0          0      0   
2       1000002  1033939  206.608239  2.492635e+08     0          0      0   
3       1000003  1044327  215.033456  2.478547e+08     0          0      0   
4       1000004  1063942  408.790952  2.466977e+08     0          0      0   
...         ...      ...         ...           ...   ...        ...    ...   
298272  8426503    -9999    1.000000  9.400000e+05     1          1      0   
298273  8426948  6010575    1.000000  2.400000e+05     1          0      0   
298274  8426992    -9999    1.000000  6.900000e+05     1          1      0   
298275  8427034    -9999    1.000000  1.900000e+05     1          1      0   
298276  8427689  3044946    1.000000  1.171261e+11     1          0      0   

                 S_0         S_max  Coeff  power  lake_type  
0

<xarray.Dataset>
Dimensions:    (n: 298277)
Coordinates:
    ID         (n) float64 ...
Dimensions without coordinates: n
Data variables:
    ID_next    (n) float64 ...
    length     (n) float64 ...
    area       (n) float64 ...
    lake       (n) float64 ...
    endorheic  (n) float64 ...
    slope      (n) float64 ...
    S_0        (n) float64 ...
    S_max      (n) float64 ...
    Coeff      (n) float64 ...
    power      (n) float64 ...
    lake_type  (n) float64 ...
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Fri Jul 14 14:50:18 2023
    source:       Written by test script of utilities (https://github.com/She...

In [5]:
ds

<xarray.Dataset>
Dimensions:    (n: 298277)
Coordinates:
    ID         (n) int64 1000000 1000001 1000002 ... 8426992 8427034 8427689
Dimensions without coordinates: n
Data variables:
    ID_next    (n) int64 1035282 1028025 1033939 1044327 ... -9999 -9999 3044946
    length     (n) float64 616.3 192.2 206.6 215.0 408.8 ... 1.0 1.0 1.0 1.0 1.0
    area       (n) float64 2.483e+08 2.492e+08 2.493e+08 ... 1.9e+05 1.171e+11
    lake       (n) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
    endorheic  (n) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 1 1 0 1 1 0
    slope      (n) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0
    S_0        (n) float64 1e+06 1e+06 1e+06 1e+06 ... 2.1e+06 3.1e+05 8.41e+12
    S_max      (n) float64 1.242e+09 1.247e+09 1.247e+09 ... 1.26e+06 8.996e+12
    Coeff      (n) float64 0.01 0.01 0.01 0.01 0.01 ... 0.0 0.01 0.0 0.0 0.01
    power      (n) float64 1.5 1.5 1.5 1.5 1.5 1.5 ... 1.5 1.5 1.5 1.5 1.5 1.5
    lake_type  (n) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 0 0 1 0 0 1
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Fri Jul 14 14:43:10 2023
    source:       Written by test script of utilities (https://github.com/She...

In [ ]:
import xarray as xr

ds = xr.open_dataset ('../data/Network_topology_HDMA_HydroLake_v1.nc')

print(ds)


ID = ds['ID'][:]
idx = np.where(ID== 7000016) # is the outflow of Lake Difenbaker
idx = np.array(idx)
# print(idx)
idx = idx[0].item()
print(idx)
print(ds['lake_type'][idx])
print(ds['Coeff'][idx])
print(ds['ID_next'][idx])
print(ds['Coeff'][idx])
print(ds['endorheic'][idx])
print(ds['S_0'][idx])
print(ds['S_max'][idx])
print(ds['area'][idx])
print(ds['slope'][idx])
